In [ ]:
import numpy as np
from sklearn import cluster, covariance
import matplotlib

import matplotlib.pyplot as plt

import os
import scipy
from scipy import interpolate


import matplotlib.cm as cm
import scipy.stats as stats

import pandas as pd

import scipy.io as sio

In [ ]:
sorted_file = []
sorted_file.append('/mnt/home/llc/TC/m-CB/goodtraces.txt')#### goodtraces location..need change..

In [ ]:
print(sorted_file)

In [ ]:
### load data
###for goodtraces.txt 

# filename_data = '/mnt/home/llc2/PZC/第1-7组goodtrace/goodtraces.txt'  ###### goodtraces location...
data = pd.DataFrame()
for filename_data in sorted_file:
    
    datatemp = pd.read_csv(filename_data, header=None, delim_whitespace=True)
    data = pd.concat((data, datatemp))
    
data = np.array(data.iloc[:, :], copy=False)
mdata, ndata = data.shape

print(mdata, ndata)

In [ ]:
###count trace ,,split the goodtrace into single trace ---list DataDistandCondu,,,
### according the distance gap to split traces 

distanceGap = 0.1
NumTraces = 0
DistDiff = np.diff(data[:, 0][:])
TraceIndex = np.where(np.abs(DistDiff) > distanceGap)[0] + 1 #####cut by distance gap == 1 ,,,,may be need change !!
NumTraces = TraceIndex.shape[0] + 1
TraceIndex = TraceIndex.tolist()
TraceIndex.append(mdata)

DataDistandCondu = []
DataDistandCondu.append(data[:TraceIndex[0], :])
for i in range(NumTraces-1):
    DataDistandCondu.append(data[TraceIndex[i]:TraceIndex[i+1], :])

print(len(DataDistandCondu)) ####check the number of traces

In [ ]:
ConduTraces = np.array(DataDistandCondu)

In [ ]:
del DataDistandCondu, TraceIndex, DistDiff, data

In [ ]:
import sys
sys.path.append('./DataPro/')

In [ ]:
import metrics as DataMetrics
import ClusteringMethods as clm
import FigPlot as Fplot
import DataSelection as DataSel

## global parameters

In [ ]:
dataHistAll, dataHistAll_x = clm.getHist1D(ConduTraces, 900, ROI=(-8, 0.99))

# show curve and 2d and 1d histograms

In [ ]:
Fplot.plotCurve(ConduTraces[60:69])

In [ ]:
Fplot.plot(ConduTraces, 4)

In [ ]:
rangeH=((-0.2, 1.5), (-7, 0.3))

In [ ]:
vmax = 1.0
vmin = 0.0

In [ ]:
ZeroAllTmp = np.zeros(NumTraces).astype(np.int)
Fplot.plot2dCloudofClusters_2(ZeroAllTmp, ConduTraces,rangeH=rangeH, vmax=vmax, vmin=vmin, save=0, filename_data=None)

In [ ]:
Fplot.plot_hist3Line(rangeH[1][0], rangeH[1][1], ZeroAllTmp, dataHistAll, dataHistAll_x)

# Clustering histogram 1d

# Note:, set the region of interest..ROI

In [ ]:
lowlogG, highlogG = -7, -1 ######

In [ ]:
labelList_5_1_SVD = clm.clusteringSVD(lowlogG, highlogG, dataHistAll_x, dataHistAll, 17)

In [ ]:
CH_index_SVD = clm.plotCHI(labelList_5_1_SVD)

In [ ]:
# Kopt = np.argmax(CH_index_SVD)
Kopt = np.where(CH_index_SVD == CH_index_SVD.max())[0]
print(Kopt.shape, Kopt)

In [ ]:
Fplot.plot2dCloudofClusters_2(labelList_5_1_SVD[0][Kopt[0]], ConduTraces,rangeH = rangeH, vmax=vmax, vmin=vmin, save=0, filename_data=None)

In [ ]:
Fplot.plot_hist3Line(-7,0.3,labelList_5_1_SVD[0][Kopt[0]], dataHistAll, dataHistAll_x)

In [ ]:
Fplot.TimeSeq(labelList_5_1_SVD[0][Kopt[0]], Kopt[0]+2)

In [ ]:

def GetResult(CH_index_SVD, labelList_5_1_SVD,ConduTraces=ConduTraces,
              dataHistAll=dataHistAll,dataHistAll_x=dataHistAll_x):
    Kopt = np.argmax(CH_index_SVD)

    Fplot.plot2dCloudofClusters_2(labelList_5_1_SVD[0][Kopt], ConduTraces,rangeH = rangeH, vmax=vmax, vmin=vmin, save=0, filename_data=None)

    Fplot.plot_hist3Line(-7,0.3,labelList_5_1_SVD[0][Kopt], dataHistAll, dataHistAll_x)

    Fplot.TimeSeq(labelList_5_1_SVD[0][Kopt], Kopt+2)
    return Kopt

# Kmeans with different distance

In [ ]:
# lowlogG, highlogG = -5, -1
cbins = np.where((dataHistAll_x >= lowlogG) & (dataHistAll_x <= highlogG))[0]
dataHistAll_6 = dataHistAll[:, cbins]

In [ ]:
def FindConst(zeroAll):
    index = []
    for i in range(len(zeroAll)):
        if(np.all(zeroAll[i] - zeroAll[i].mean() == 0) ):
            index.append(i)
    return np.array(index)

In [ ]:
def FindConstStd(zeroAll):
    index = []
    for i in range(len(zeroAll)):
        if(np.abs(zeroAll[i].std()) < 0.00001):
            index.append(i)
    return np.array(index)

In [ ]:
from sklearn import preprocessing


In [ ]:
FindConst(dataHistAll_6) ###DataMetrics.FindConst(dataHistAll)

In [ ]:
FindConstStd(dataHistAll_6)

In [ ]:
dataHistAll_6Scale = preprocessing.scale(dataHistAll_6, axis=1)####each sample scaling

In [ ]:
labelList_5_1KmScale = clm.clusteringBtweenKmeans(lowlogG, highlogG, dataHistAll_x[cbins], dataHistAll_6Scale, 9)

In [ ]:
VNC_Index, davies_bouldin_Index, calinski_harabasz_Index,Silhouette_Index = clm.plotIndex(labelList_5_1KmScale)

In [ ]:
testChi = clm.plotCHI(labelList_5_1KmScale,)# 'euclidean')

In [ ]:
GetResult(testChi, labelList_5_1KmScale)

In [ ]:
testChi2 = clm.plotCHI(labelList_5_1KmScale, 'euclidean')

### above prescale : kmeans may be equal to spectral clustering...!!!!

#### below new distance test:::

In [ ]:
labelListKmeans = []
for i in range(2, 9):
    CenterTmp, labes_KmeansTmp = clm.find_centers(dataHistAll_6, i, DataMetrics.DistanceFunCorre)###random,so not stable..
    labelListKmeans.append(labes_KmeansTmp)

In [ ]:
for i in range(0, 7):
    Fplot.plotHistofClusters_perTrace(labelListKmeans[i],lowlogG, highlogG,dataHistAll, dataHistAll_x)

In [ ]:
labelListKmeansCHI = clm.plotCHI([labelListKmeans, dataHistAll_6])

In [ ]:
OptNum = GetResult(labelListKmeansCHI, [labelListKmeans, 1])

In [ ]:
dataHistAll100, dataHistAll_x100 = clm.getHist1D(ConduTraces)

In [ ]:
Fplot.plotHistofClusters_perTrace(labelListKmeans[OptNum],-7, 0.3,dataHistAll100, dataHistAll_x100)#####do not like a peak of each cluster

# choose ROI from original data

In [ ]:
DataChosen = DataSel.TracesSelect(-2, -6, ConduTraces,0.005)

In [ ]:
DataChosen = np.array(DataChosen)

In [ ]:
print(DataChosen[0].shape, NumTraces)####check if filter some traces?

In [ ]:
DataChosen[0] = np.delete(ConduTraces, DataChosen[1])

In [ ]:
Fplot.plot(DataChosen[0], 1653)

In [ ]:
# DataChosenRd = DataSel.RelatvieDist(DataChosen[0])##
###could change the ditance value....may not needed...be careful..

In [ ]:
Fplot.plotCurve(DataChosen[0][50:80])

In [ ]:
HistDatSel, HistDataSel_x = clm.getHist1D(DataChosen[0], cbins=400, ROI=(lowlogG, highlogG))

In [ ]:
labelList_5_1_SVD_Sel = clm.clusteringSVD(lowlogG, highlogG, HistDataSel_x, HistDatSel, 17)

In [ ]:
DataChosenCHI = clm.plotCHI(labelList_5_1_SVD_Sel)

In [ ]:
OptNumChoosn = GetResult(DataChosenCHI, labelList_5_1_SVD_Sel,DataChosen[0],HistDatSel, HistDataSel_x )

# multi plateau test
ggggg

In [ ]:
Fplot.plot2dCloudofClusters_2(np.zeros(len(DataChosen[0])), 
                              DataChosen[0], vmax=vmax, rangeH=rangeH)

# note:set distance range;;;;,,,, just test!!

In [ ]:
# data_hist_ibins[0]
distRange = [0.01, 1.5]
dataHistAllAug, dataHistAllAugScale = clm.getNewHist(ConduTraces, distRange, [lowlogG, highlogG])

In [ ]:
FindConst(dataHistAllAug)

In [ ]:
labellistAug_Km2d = clm.clusteringBtweenKmeans_2d(dataHistAllAug, 17)

In [ ]:
labellistAug_Km2dCHI = clm.plotCHI(labellistAug_Km2d)

In [ ]:
OptNumKm2dCHI = GetResult(labellistAug_Km2dCHI, labellistAug_Km2d)

In [ ]:
labellistAugScale_Km2d = clm.clusteringBtweenKmeans_2d(dataHistAllAugScale, 17)

In [ ]:
labellistAugScale_Km2dCHI = clm.plotCHI(labellistAugScale_Km2d)

In [ ]:
OptNum_SK = GetResult(labellistAugScale_Km2dCHI, labellistAugScale_Km2d)

In [ ]:
labellistAug = clm.clusteringBtweenSP_2d(dataHistAllAug, 17)

In [ ]:
labellistAugScale = clm.clusteringBtweenSP_2d(dataHistAllAugScale, 17)

In [ ]:
labellistAugCHI = clm.plotCHI(labellistAug)

In [ ]:
labellistAugCHIScale = clm.plotCHI(labellistAugScale)

In [ ]:
OptNum_AS = GetResult(labellistAugCHIScale, labellistAugScale)

In [ ]:
OptNum_A = GetResult(labellistAugCHI, labellistAug)

In [ ]:
for i in range(7):
    Fplot.plotHistofClusters_perTrace(labellistAug[0][i], -7, 0.3, dataHistAll, dataHistAll_x)

In [ ]:
Fplot.plot_hist3Line(lowlogG, 0.3, labellistAug[0][3], dataHistAll, dataHistAll_x)